In [1]:
# --- imports
import os
import ast
import numpy as np
import pandas as pd
import joblib
from typing import Optional, Tuple, Iterable


# ---- Pickle compatibility shims for TransformedTargetRegressor ----
def _register_pickle_shims():
    """
    Ensure the names used during training exist in the current __main__ module,
    so joblib/pickle can resolve them when unpickling.
    """
    import sys
    import numpy as np

    main = sys.modules.get("__main__")

    if not hasattr(main, "y_to_z"):
        def y_to_z(y, eps=1e-6):
            y = np.asarray(y, dtype=float)
            # clamp to (-1, 1) to keep atanh finite
            y = np.clip(y, -1 + eps, 1 - eps)
            return np.arctanh(y)
        setattr(main, "y_to_z", y_to_z)

    if not hasattr(main, "z_to_y"):
        def z_to_y(z):
            z = np.asarray(z, dtype=float)
            return np.tanh(z)
        setattr(main, "z_to_y", z_to_y)


# ----------------------------
# Given helper (from you)
# ----------------------------
def concat_years(start_year: int, end_year: int, base_dir: str, file_type: str = "10K"):
    """Concatenate yearly parquet files into a single DataFrame"""
    dfs = []
    for year in range(start_year, end_year + 1):
        path = os.path.join(base_dir, f"{year}_mgmt_training_{file_type}.parquet")
        if os.path.exists(path):
            year_df = pd.read_parquet(path, engine="fastparquet")
            dfs.append(year_df)
        else:
            print(f"⚠️ Skipping missing file: {path}")
    return pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()

# ----------------------------
# Config
# ----------------------------
DATA_BASE_DIR = "../../data/training_data_surprise_model"
MODEL_DIR     = "../../data/rolling_window_results/models"
FILE_TYPE     = "10K"    # change to "10Q" if needed

RAW_COL = "mgmt_pred_raw"
CAL_COL = "mgmt_pred_cal"       # calibrated (isotonic) result
FINAL_COL = "mgmt_pred"         # alias to the calibrated score

# ----------------------------
# Utilities
# ----------------------------
EMBEDDING_SIZE = 768
ZERO_VEC = np.zeros(EMBEDDING_SIZE, dtype=np.float32)

def _to_vec768(x) -> np.ndarray:
    """
    Robustly convert the mgmt_embedding cell into a 768-dim numpy vector.
    Falls back to all-zeros if empty/NaN/incorrect size.
    Handles list/np.ndarray or a stringified list.
    """
    if x is None or (isinstance(x, float) and np.isnan(x)):
        return ZERO_VEC

    # If it's a string like "[...]", try parsing
    if isinstance(x, str):
        try:
            x = ast.literal_eval(x)
        except Exception:
            return ZERO_VEC

    arr = np.asarray(x, dtype=np.float32)
    if arr.ndim != 1 or arr.size == 0:
        return ZERO_VEC

    # pad/trim to 768 defensively
    if arr.size < EMBEDDING_SIZE:
        padded = np.zeros(EMBEDDING_SIZE, dtype=np.float32)
        padded[:arr.size] = arr
        return padded
    elif arr.size > EMBEDDING_SIZE:
        return arr[:EMBEDDING_SIZE]
    else:
        return arr

def _stack_embeddings(df: pd.DataFrame, col: str = "mgmt_embedding") -> np.ndarray:
    """
    Convert the column of embeddings into a 2D array (n, 768).
    Empty/missing entries become zeros.
    """
    return np.stack(df[col].apply(_to_vec768).values, axis=0)

def _model_paths(year: int, model_dir: str = MODEL_DIR) -> Tuple[str, str]:
    main_path = os.path.join(model_dir, f"model_test_{year}.pkl")
    iso_path  = os.path.join(model_dir, f"iso_model_test_{year}.pkl")
    return main_path, iso_path

def load_models_for_year(year: int, model_dir: str = MODEL_DIR):
    import os, joblib

    main_path = os.path.join(model_dir, f"model_test_{year}.pkl")
    iso_path  = os.path.join(model_dir, f"iso_model_test_{year}.pkl")

    if not os.path.exists(main_path):
        raise FileNotFoundError(f"Main model not found: {main_path}")

    _register_pickle_shims()               # <-- make functions resolvable
    main_model = joblib.load(main_path)    # unpickle succeeds now

    iso_model = joblib.load(iso_path) if os.path.exists(iso_path) else None
    if iso_model is None:
        print(f"ℹ️ No isotonic model found for {year} at {iso_path}; using raw scores only.")

    return main_model, iso_model


# ----------------------------
# Inference for a single year
# ----------------------------
def score_year(
    year: int,
    base_dir: str = DATA_BASE_DIR,
    file_type: str = FILE_TYPE,
    model_dir: str = MODEL_DIR,
    save_path: Optional[str] = None,
) -> pd.DataFrame:
    """
    - Loads the single-year dataframe
    - Builds X from mgmt_embedding (zeros for missing/empty)
    - Runs main model -> optional isotonic calibration
    - Adds columns:
        mgmt_pred_raw : raw model output
        mgmt_pred_cal : calibrated output (if calibrator exists; else equals raw)
        mgmt_pred     : alias to calibrated output
    - Optionally saves to `save_path` (parquet if endswith .parquet, otherwise pickle)
    """
    df = concat_years(year, year, base_dir, file_type=file_type)
    if df.empty:
        print(f"⚠️ No data for year {year} in {base_dir} (type={file_type}).")
        return df

    # Prepare features
    X = _stack_embeddings(df, col="mgmt_embedding")

    # Load models
    main_model, iso_model = load_models_for_year(year, model_dir=model_dir)

    # Predict
    raw = main_model.predict(X)
    if iso_model is not None:
        cal = iso_model.predict(raw)
    else:
        cal = raw

    # Attach to dataframe
    df[RAW_COL] = raw
    df[CAL_COL] = cal
    df[FINAL_COL] = cal  # convenience alias

    # Optional save
    if save_path:
        if save_path.endswith(".parquet"):
            df.to_parquet(save_path, index=False, engine="fastparquet")
        else:
            df.to_pickle(save_path)
        print(f"✅ Saved scored dataframe to {save_path}")

    return df

# ----------------------------
# Inference over a range of years (concatenate)
# ----------------------------
def score_years(
    start_year: int,
    end_year: int,
    base_dir: str = DATA_BASE_DIR,
    file_type: str = FILE_TYPE,
    model_dir: str = MODEL_DIR,
    per_year_save_dir: Optional[str] = None,
    return_concat: bool = True,
) -> Optional[pd.DataFrame]:
    """
    Loop years, score each year with its own model + calibrator, optionally
    save each year’s result, and optionally return a single concatenated DataFrame.
    """
    all_dfs = []
    for y in range(start_year, end_year + 1):
        try:
            save_path = None
            if per_year_save_dir:
                os.makedirs(per_year_save_dir, exist_ok=True)
                # choose your preferred format:
                save_path = os.path.join(per_year_save_dir, f"{y}_scored.parquet")
            df_y = score_year(
                year=y,
                base_dir=base_dir,
                file_type=file_type,
                model_dir=model_dir,
                save_path=save_path,
            )
            if not df_y.empty and return_concat:
                all_dfs.append(df_y)
        except FileNotFoundError as e:
            print(f"❌ {e}")
        except Exception as e:
            print(f"❌ Error scoring year {y}: {e}")

    if return_concat and all_dfs:
        concat_df = pd.concat(all_dfs, ignore_index=True)
        return concat_df
    return None

In [2]:
# Collect all years into one big DataFrame
all_preds = []

for year in range(2009, 2026):
    try:
        df_pred = score_year(year)   # <-- use the loop variable
        if not df_pred.empty:
            all_preds.append(df_pred.assign(year_scored=year))  # optional tag column
    except Exception as e:
        print(f"❌ Error scoring {year}: {e}")

final_pred_df = pd.concat(all_preds, ignore_index=True) if all_preds else pd.DataFrame()

print("Final shape:", final_pred_df.shape)


Final shape: (52699, 9)


In [3]:
import pandas as pd
import numpy as np

def _pick_gvkey(df):
    """Handle either 'gvkey' or 'gv_key'."""
    if "gvkey" in df.columns:
        return "gvkey"
    if "gv_key" in df.columns:
        return "gv_key"
    raise KeyError("Neither 'gvkey' nor 'gv_key' found.")

def _to_intish(series):
    """Make gvkey comparable (handles floats/strings/NaNs)."""
    # keep NA as <NA> using pandas nullable Int64
    return pd.to_numeric(series, errors="coerce").astype("Int64")

def _to_month_end(dt_series):
    """
    Convert a date-like series to actual month-end timestamps.
    Works for 'YYYY-MM-DD', 'YYYY-MM', ints like 20050228, etc.
    """
    # try direct datetime parsing
    dt = pd.to_datetime(dt_series, errors="coerce", format=None)

    # If something is still NaT, try numeric yyyymmdd
    mask_nat = dt.isna()
    if mask_nat.any():
        dt2 = pd.to_datetime(dt_series[mask_nat].astype(str), errors="coerce", format="%Y%m%d")
        dt.loc[mask_nat] = dt2

    # If we got 'YYYY-MM' it will parse to first of month → push to month-end.
    # If already a proper date, MonthEnd(0) will snap to that month’s end.
    return (dt + pd.offsets.MonthEnd(0))

def merge_quant_with_preds(
    quant_df: pd.DataFrame,
    preds_df: pd.DataFrame,
    pred_col: str = "mgmt_pred",
) -> pd.DataFrame:
    """
    Left-join quant data with predictions on (gvkey, month-end date).
    - quant_df['date'] is month-end (but we normalize anyway)
    - preds_df['date'] is monthly (e.g., 'YYYY-MM'); we normalize to month-end
    - missing predictions default to 0
    - if multiple predictions exist for the same (gvkey, month), we average them
    Returns a new DataFrame = quant_df with one new column: pred_col
    """

    # Resolve gvkey column names
    q_gv = _pick_gvkey(quant_df)
    p_gv = _pick_gvkey(preds_df)

    quant_df["_gv_"] = _to_intish(quant_df[q_gv])
    preds_df["_gv_"] = _to_intish(preds_df[p_gv])

    quant_df["_eom_"] = _to_month_end(quant_df["date"])
    preds_df["_eom_"] = _to_month_end(preds_df["date"])

    # Reduce predictions to one row per (gvkey, month). If duplicates, take mean.
    p_reduced = (
        preds_df[["_gv_", "_eom_", pred_col]]
        .groupby(["_gv_", "_eom_"], dropna=False, as_index=False)
        .mean(numeric_only=True)
    )

    # Left-merge and default missing preds to zero
    merged = quant_df.merge(p_reduced, on=["_gv_", "_eom_"], how="left", suffixes=("", "_predsrc"))
    if pred_col not in merged.columns:
        raise Exception("prediction column was absent (edge case), create it as zeros")
    
    merged[pred_col] = merged[pred_col].fillna(0.0)

    # Clean helper columns
    merged = merged.drop(columns=["_gv_", "_eom_"])

    return merged


In [4]:
# 1) Your primary quant data (with last-day-of-month dates)
quant_df = pd.read_parquet("../../data/ret_sample.parquet")  # has 'gvkey' (or 'gv_key') and 'date'

# 2) Your predictions from earlier step (per year or concatenated)
#    Must contain: 'gvkey' (or 'gv_key'), 'date' (monthly), and 'mgmt_pred'  # or the concatenated output from score_years(...)

# 3) Merge (left-join) and default missing preds to 0
merged_df = merge_quant_with_preds(quant_df, final_pred_df, pred_col="mgmt_pred")

In [5]:
merged_df

,id,date,ret_eom,gvkey,iid,excntry,stock_ret,year,month,char_date,...,prc_highprc_252d,corr_1260d,betabab_1260d,rmax5_rvol_21d,age,qmj,qmj_prof,qmj_growth,qmj_safety,mgmt_pred
0,comp_001081_01C,2005-02-28,20050228,1081.0,01C,CAN,-0.143457,2005,2,20050131,...,0.672204,0.387781,0.845865,0.805580,541,-1.508294,-0.994164,-0.832048,-1.017248,0.0
1,comp_001096_01C,2005-02-28,20050228,1096.0,01C,CAN,0.028077,2005,2,20050131,...,0.937664,0.245148,0.456872,0.923214,517,-0.706080,-0.247574,-0.155802,-0.485635,0.0
2,comp_001117_02,2005-02-28,20050228,1117.0,02,USA,-0.168627,2005,2,20050131,...,0.708333,0.124188,0.863334,0.898113,373,1.344458,1.601108,1.612067,-0.566631,0.0
3,comp_001166_01W,2005-02-28,20050228,1166.0,01W,NLD,0.086271,2005,2,20050131,...,0.676545,0.560895,1.560202,1.342814,289,-1.355529,-0.904719,-0.999531,-1.231687,0.0
4,comp_001186_01C,2005-02-28,20050228,1186.0,01C,CAN,0.149056,2005,2,20050131,...,0.774557,0.174888,0.399060,0.777183,385,1.123762,0.154734,1.196690,0.939661,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6401409,comp_367287_01W,2025-06-30,20250630,367287.0,01W,HKG,-0.261016,2025,6,20250530,...,NaN,NaN,NaN,NaN,41,NaN,NaN,NaN,NaN,0.0
6401410,comp_367313_01W,2025-06-30,20250630,367313.0,01W,DEU,-0.573456,2025,6,20250530,...,NaN,NaN,NaN,NaN,29,NaN,NaN,NaN,NaN,0.0
6401411,comp_367333_01W,2025-06-30,20250630,367333.0,01W,ITA,0.463789,2025,6,20250530,...,NaN,NaN,NaN,NaN,29,NaN,NaN,NaN,NaN,0.0
6401412,comp_367334_01W,2025-06-30,20250630,367334.0,01W,ITA,-0.215849,2025,6,20250530,...,NaN,NaN,NaN,NaN,5,NaN,NaN,NaN,NaN,0.0


In [6]:
merged_df[merged_df['mgmt_pred'] != 0]

,id,date,ret_eom,gvkey,iid,excntry,stock_ret,year,month,char_date,...,prc_highprc_252d,corr_1260d,betabab_1260d,rmax5_rvol_21d,age,qmj,qmj_prof,qmj_growth,qmj_safety,mgmt_pred
1137693,comp_012717_01,2009-01-30,20090131,12717.0,01,USA,-0.279070,2009,1,20081231,...,0.288591,0.072589,0.186152,2.996967,312,-0.164377,-0.515303,1.622090,-1.284017,-0.220582
1157783,crsp_10025,2009-01-30,20090131,11903.0,01,USA,-0.184300,2009,1,20081231,...,0.540258,0.403823,0.688997,0.756965,288,-0.391072,-0.317333,0.971809,-1.180804,0.037694
1157852,crsp_10659,2009-01-30,20090131,12576.0,02,USA,-0.069767,2009,1,20081231,...,0.826662,0.432514,0.540778,2.040936,288,0.708084,1.270789,-0.798401,0.798270,0.108810
1157889,crsp_11132,2009-01-30,20090131,13013.0,01,USA,-0.130736,2009,1,20081231,...,0.767090,0.331934,0.484224,2.021043,276,0.629508,0.311511,-0.849881,1.661499,0.108810
1157892,crsp_11154,2009-01-30,20090131,13200.0,01,USA,-0.184615,2009,1,20081231,...,0.151869,0.481066,1.823525,2.117733,276,-1.245469,0.127370,-1.016967,-1.503433,0.053053
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6350258,comp_176999_01,2025-05-30,20250531,176999.0,01,USA,0.027049,2025,5,20250430,...,0.934791,0.494089,0.523430,2.255789,220,0.586731,1.109230,0.694671,-0.811128,-0.136858
6350263,comp_177111_01,2025-05-30,20250531,177111.0,01,USA,0.015056,2025,5,20250430,...,0.229353,0.525217,1.667690,1.453085,256,-1.659962,-1.241333,-1.670756,-1.284643,0.008019
6350439,comp_185881_01,2025-05-30,20250531,185881.0,01,USA,-0.114731,2025,5,20250430,...,0.645269,0.341899,0.617212,1.493744,232,1.027743,1.076205,0.944475,-0.261793,0.141677
6350461,comp_186501_01,2025-05-30,20250531,186501.0,01,USA,0.175084,2025,5,20250430,...,0.500843,0.304812,0.839874,1.445777,232,-1.260585,-0.711487,-0.164223,-1.360463,0.008019


In [7]:
merged_df = merged_df[['id', 'date', 'gvkey', 'mgmt_pred']]
# merged_df.to_parquet("./mgmt_pred_feature.parquet")

In [8]:
import pandas as pd
import numpy as np

def add_ffilled_pred(
    df: pd.DataFrame,
    id_col: str = "id",
    date_col: str = "date",
    pred_col: str = "mgmt_pred",
    out_col: str = "mgmt_pred_ffill",
    max_stale_months: int | None = None,
) -> pd.DataFrame:
    """
    Forward-fill `pred_col` per `id_col` (sorted by `date_col`).
    If `max_stale_months` is set, invalidate filled values that are older than that cap.
    """
    df = df.copy()

    # normalize date -> month-end and sort
    dt = pd.to_datetime(df[date_col], errors="coerce")
    df["_eom_"] = (dt + pd.offsets.MonthEnd(0))
    df = df.sort_values([id_col, "_eom_"])

    # ffill per id (zeros should already be NaN before calling; if not, do it here)
    df[out_col] = df.groupby(id_col, observed=True)[pred_col].ffill()

    if max_stale_months is not None:
        # last true-pred date per row (forward-filled within id)
        last_pred_dt = (
            df.groupby(id_col, observed=True)[["_eom_", pred_col]]
              .apply(lambda g: g["_eom_"].where(g[pred_col].notna()).ffill())
              .reset_index(level=0, drop=True)
        )
        df["_last_pred_dt"] = last_pred_dt

        # Compute month index = year*12 + month; handles NaT -> <NA>
        eom_idx  = (df["_eom_"].dt.year * 12 + df["_eom_"].dt.month).astype("Int64")
        last_idx = (df["_last_pred_dt"].dt.year * 12 + df["_last_pred_dt"].dt.month).astype("Int64")

        months_since = (eom_idx - last_idx)  # dtype Int64 with <NA>s

        # Invalidate stale fills: only apply where we actually have a prior true pred
        stale_mask = months_since.notna() & (months_since > max_stale_months)
        df.loc[stale_mask, out_col] = np.nan

        df.drop(columns=["_last_pred_dt"], inplace=True)

    df.drop(columns=["_eom_"], inplace=True)
    return df


In [9]:
merged_df["mgmt_pred"] = merged_df["mgmt_pred"].replace(0, np.nan)

merged_df = add_ffilled_pred(
    merged_df,
    id_col="id",
    date_col="date",
    pred_col="mgmt_pred",
    out_col="mgmt_pred_ffill",
    max_stale_months=6,   # or None
)

/var/folders/66/lykx91p150q4wl3w7w5ypwdh0000gn/T/ipykernel_31030/4055150072.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df["mgmt_pred"] = merged_df["mgmt_pred"].replace(0, np.nan)


In [10]:
merged_df.dropna(subset='mgmt_pred_ffill')

,id,date,gvkey,mgmt_pred,mgmt_pred_ffill
6262266,comp_001050_01,2025-02-28,1050.0,-0.098073,-0.098073
6290191,comp_001050_01,2025-03-31,1050.0,NaN,-0.098073
6318092,comp_001050_01,2025-04-30,1050.0,NaN,-0.098073
6345917,comp_001050_01,2025-05-30,1050.0,NaN,-0.098073
6373677,comp_001050_01,2025-06-30,1050.0,NaN,-0.098073
...,...,...,...,...,...
5965640,crsp_93436,2024-03-28,184996.0,NaN,0.058653
5995905,crsp_93436,2024-04-30,184996.0,NaN,0.058653
6026109,crsp_93436,2024-05-31,184996.0,NaN,0.058653
6056298,crsp_93436,2024-06-28,184996.0,NaN,0.058653


In [11]:
# merged_df = merged_df.drop(columns="mgmt_pred")
merged_df['mgmt_pred'] = merged_df['mgmt_pred_ffill']
merged_df.drop(columns=['mgmt_pred_ffill'])

,id,date,gvkey,mgmt_pred
6262264,comp_001004_01,2025-02-28,1004.0,NaN
6290189,comp_001004_01,2025-03-31,1004.0,NaN
6318090,comp_001004_01,2025-04-30,1004.0,NaN
6345915,comp_001004_01,2025-05-30,1004.0,NaN
6373675,comp_001004_01,2025-06-30,1004.0,NaN
...,...,...,...,...
6116621,crsp_93436,2024-08-30,184996.0,NaN
6146720,crsp_93436,2024-09-30,184996.0,NaN
6176783,crsp_93436,2024-10-31,184996.0,NaN
6206837,crsp_93436,2024-11-29,184996.0,NaN


In [12]:
merged_df = merged_df.drop(columns='mgmt_pred_ffill')


In [13]:
merged_df

,id,date,gvkey,mgmt_pred
6262264,comp_001004_01,2025-02-28,1004.0,NaN
6290189,comp_001004_01,2025-03-31,1004.0,NaN
6318090,comp_001004_01,2025-04-30,1004.0,NaN
6345915,comp_001004_01,2025-05-30,1004.0,NaN
6373675,comp_001004_01,2025-06-30,1004.0,NaN
...,...,...,...,...
6116621,crsp_93436,2024-08-30,184996.0,NaN
6146720,crsp_93436,2024-09-30,184996.0,NaN
6176783,crsp_93436,2024-10-31,184996.0,NaN
6206837,crsp_93436,2024-11-29,184996.0,NaN


In [14]:
merged_df.to_parquet("./mgmt_pred_feature_with_nan.parquet")

In [15]:
merged_df.dropna(subset='mgmt_pred')

,id,date,gvkey,mgmt_pred
6262266,comp_001050_01,2025-02-28,1050.0,-0.098073
6290191,comp_001050_01,2025-03-31,1050.0,-0.098073
6318092,comp_001050_01,2025-04-30,1050.0,-0.098073
6345917,comp_001050_01,2025-05-30,1050.0,-0.098073
6373677,comp_001050_01,2025-06-30,1050.0,-0.098073
...,...,...,...,...
5965640,crsp_93436,2024-03-28,184996.0,0.058653
5995905,crsp_93436,2024-04-30,184996.0,0.058653
6026109,crsp_93436,2024-05-31,184996.0,0.058653
6056298,crsp_93436,2024-06-28,184996.0,0.058653


In [16]:

merged_df["mgmt_pred"] = merged_df["mgmt_pred"].replace(np.nan, 0)

merged_df.to_parquet("./mgmt_pred_feature.parquet")


In [17]:
merged_df

,id,date,gvkey,mgmt_pred
6262264,comp_001004_01,2025-02-28,1004.0,0.0
6290189,comp_001004_01,2025-03-31,1004.0,0.0
6318090,comp_001004_01,2025-04-30,1004.0,0.0
6345915,comp_001004_01,2025-05-30,1004.0,0.0
6373675,comp_001004_01,2025-06-30,1004.0,0.0
...,...,...,...,...
6116621,crsp_93436,2024-08-30,184996.0,0.0
6146720,crsp_93436,2024-09-30,184996.0,0.0
6176783,crsp_93436,2024-10-31,184996.0,0.0
6206837,crsp_93436,2024-11-29,184996.0,0.0


In [18]:
merged_df[merged_df['mgmt_pred'] != 0 ]

,id,date,gvkey,mgmt_pred
6262266,comp_001050_01,2025-02-28,1050.0,-0.098073
6290191,comp_001050_01,2025-03-31,1050.0,-0.098073
6318092,comp_001050_01,2025-04-30,1050.0,-0.098073
6345917,comp_001050_01,2025-05-30,1050.0,-0.098073
6373677,comp_001050_01,2025-06-30,1050.0,-0.098073
...,...,...,...,...
5965640,crsp_93436,2024-03-28,184996.0,0.058653
5995905,crsp_93436,2024-04-30,184996.0,0.058653
6026109,crsp_93436,2024-05-31,184996.0,0.058653
6056298,crsp_93436,2024-06-28,184996.0,0.058653
